In [1]:
from datetime import datetime
from pathlib import Path

import pandas as pd
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [2]:
import sys  
sys.path.insert(0, 'data')

In [4]:
input_data = Path("data")


In [6]:
data = pd.read_csv("data/Waste_Collection___Diversion_Report__daily_.csv")
# data = pd.read_csv(input_data / "clean_waste_data.csv")
data['Report Date'] = data['Report Date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
data['Load Time'] = data['Load Time'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
data['Load Weight'] = data['Load Weight'].apply(lambda x: float(str(x).replace(',', '')))

In [7]:
data.head()

,Report Date,Load Type,Load Time,Load Weight,Dropoff Site,Route Type,Route Number,Load ID
0,2020-12-08,BULK,2020-12-08 15:02:00,5220.0,TDS LANDFILL,BULK,BU13,899097
1,2020-12-08,RECYCLING - SINGLE STREAM,2020-12-08 10:00:00,11140.0,TDS - MRF,RECYCLING - SINGLE STREAM,RTAU53,899078
2,2020-12-03,RECYCLING - SINGLE STREAM,2020-12-03 10:34:00,10060.0,BALCONES RECYCLING,RECYCLING - SINGLE STREAM,RHBU10,899082
3,2020-12-07,SWEEPING,2020-12-07 10:15:00,7100.0,TDS LANDFILL,SWEEPER DUMPSITES,DSS04,899030
4,2020-12-07,RECYCLING - SINGLE STREAM,2020-12-07 16:00:00,12000.0,TDS - MRF,RECYCLING - SINGLE STREAM,RMAU53,899048


In [8]:
data[["Report Date", "Load Weight", "Load Type", "Route Number"]].head()

,Report Date,Load Weight,Load Type,Route Number
0,2020-12-08,5220.0,BULK,BU13
1,2020-12-08,11140.0,RECYCLING - SINGLE STREAM,RTAU53
2,2020-12-03,10060.0,RECYCLING - SINGLE STREAM,RHBU10
3,2020-12-07,7100.0,SWEEPING,DSS04
4,2020-12-07,12000.0,RECYCLING - SINGLE STREAM,RMAU53


In [9]:
data['Route Number'].value_counts()

SDT01      11279
DSS04       6870
DA01        4323
BU13        2667
DA02        2633
           ...  
TNONMC         1
RWNON38        1
SSN18          1
PAS87          1
HNONEG         1
Name: Route Number, Length: 2221, dtype: int64

In [10]:
data[data["Route Number"] == "BU13"][["Report Date", "Load Weight", "Load Type", "Route Number"]].head()

,Report Date,Load Weight,Load Type,Route Number
0,2020-12-08,5220.0,BULK,BU13
6,2020-12-08,7700.0,BULK,BU13
7,2020-12-08,6520.0,BULK,BU13
11,2020-12-08,9260.0,BULK,BU13
57,2020-12-08,10100.0,BULK,BU13


In [ ]:
# data.groupby('Report Date')['Load Weight'].sum().plot()
df = data.groupby('Report Date')['Load Weight'].sum().reset_index()
df.columns = ['ds', 'y']
df.head()

In [ ]:
m = Prophet()
m.fit(df);
future = m.make_future_dataframe(periods=365);

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail();

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
data[data['Load Type'] == "GARBAGE COLLECTIONS"].groupby('Report Date')['Load Weight'].sum().plot();

In [ ]:
data[data['Load Type'] == "RECYCLING - SINGLE STREAM"].groupby('Report Date')['Load Weight'].sum().plot();

In [ ]:
data[data['Load Type'] == "SWEEPING"].groupby('Report Date')['Load Weight'].sum().plot();

In [ ]:
data.groupby('Report Date')['Load Weight'].sum().plot()

In [ ]:
# Adding regressors
clean_df = pd.read_csv(input_data / "clean_waste_data.csv")
clean_df['date'] = clean_df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# clean_df['Route.Type'].value_counts()
# clean_df.head()

# GARBAGE COLLECTION              201068
# RECYCLING - SINGLE STREAM        77246
# YARD TRIMMINGS                   45878
# BULK                              7898
# YARD TRIMMINGS-ORGANICS           6887

df = clean_df[clean_df['Route.Type'] == "GARBAGE COLLECTION"][['date', 'daily_weight']]
df = df.groupby('date')['daily_weight'].sum().reset_index()
df.columns = ['ds', 'y']
df.head()

In [ ]:
m = Prophet()
m.fit(df);
future = m.make_future_dataframe(periods=365);

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail();

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
df.dtypes